# Tweets preprocessing

install libraries

In [ ]:
# Data preprocessing for Crisis dataset v1.3

# This notebook is created in Google Colab, please change the paths to your file.
# Every preprocessing method is seperate. You can choose the ones that you need. 
# Write me a note if something goes wrong or you need some new preprocessing methods.

# Update from v1.0:
# Include crisis_dataset_1
# Include evaluation

# Update from v1.1:
# Seperate all the algorithms and datasets, each with two matrixes output (full)

#  Update from v1.2:
# Integrate the visualization part. This notebook has completed its mission. See LDA/NMF_runner.py for further information.
# If you would like to run this code on google colab, please check colab_LDA/NMF_runner.ipynb

# Enjoy!

In [ ]:
#!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import unicodedata
import re
import contractions
from pathlib import Path
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Load many datasets

In [ ]:
def split_dataset(tweets_df, seed = 100, TRAIN = 1, DEV = 0, TEST = 0):
    # Define the partition of train, dev and test set, make sure they sum up to 1
    PT_TRAIN = TRAIN
    PT_DEV = DEV
    PT_TEST = TEST

    # Define train, dev and test set
    tweets_df.loc[: tweets_df.shape[0] * PT_TRAIN, 'partition'] = 'train'
    tweets_df.loc[tweets_df.shape[0] * PT_TRAIN : tweets_df.shape[0] * (PT_TRAIN + PT_DEV), 'partition'] = 'dev'
    tweets_df.loc[tweets_df.shape[0] * (PT_TRAIN + PT_DEV) : tweets_df.shape[0] - 1, 'partition'] = 'test'

    # Shuffle
    final_df = tweets_df.sample(frac=1, random_state = seed).reset_index(drop=True)

    return final_df

In [ ]:
def load_documents(dataset_dir: str, dataset_text_col: str):
   # This method comes from Berk's work: src/utils.py
   # Input:
   # dataset_dir: The folder where the datasets are stored.
   # dataset_text_col: The name of the column that contains tweets.
   # (For dataset 1: tweets_text, for dataset 12: text)

   # Output:
   # One dataframe containing all the data

   dataset_data_paths = sorted([path for path in Path(dataset_dir).iterdir() if path.suffix in {'.csv', '.tsv'}])
   
   dfs = []
   labels_as_filenames = []
   topic_num = 0
   for data_path in dataset_data_paths:
      csv_delimiter = '\t' if data_path.suffix == '.tsv' else ','
      df = pd.read_csv(data_path, delimiter=csv_delimiter)
      df = df[[dataset_text_col]]
      splitted_df = split_dataset(df)
      splitted_df.loc[:, 'Topics'] = f'{topic_num}'
      topic_num += 1
      # Why do we need this?
      labels_as_filenames.extend([Path(data_path).stem] * len(df))
      dfs.append(splitted_df)
   
   merged_df = pd.concat(dfs, axis=0)
   merged_df.rename(columns={f'{dataset_text_col}':'Tweets'},inplace=True)

   return merged_df
   #assert False

In [ ]:
# Select the right dataset

# Crisis dataset No.1
#dataset_dir = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_1_original'
#save_dir = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_1'
#tweets_df = load_documents(dataset_dir, 'tweet_text')
#NUM_TOPICS = 11

# Crisis dataset No.12
dataset_dir = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12_original'
save_dir = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12'
tweets_df = load_documents(dataset_dir, 'text')
NUM_TOPICS = 4

In [ ]:
tweets_df

,Tweets,partition,Topics
0,Bay Area niggas jus felt that lil earthquake l...,train,0
1,jswartz Yea! Welcome to earthquake twitter!,train,0
2,"I think that was a light earthquake, but I won...",train,0
3,Did y all feel that earthquake?,train,0
4,Another freaking #earthquake ... what gives?! ...,train,0
...,...,...,...
1995,Malcolm Turnbull has praised technology for an...,train,3
1996,#NEWS #Cyclone Cyclone Debbie: What will happe...,train,3
1997,Watch Tropical Cyclone Debbie intensify from s...,train,3
1998,Cyclone dumps bull shark in middle of road htt...,train,3


In [ ]:
'''
# Former method

path1 = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/earthquakes_eyewitness_crowdflower_2000.tsv'
path2 = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/floods_eyewitness_crowdflower_2000.tsv'
path3 = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/forestfires_eyewitness_crowdflower_2000.tsv'
path4 = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/hurricanes_eyewitness_crowdflower_2000.tsv'

tweets_df1=pd.read_csv(path1, sep="\t")
tweets_df2=pd.read_csv(path2, sep="\t")
tweets_df3=pd.read_csv(path3, sep="\t")
tweets_df4=pd.read_csv(path4, sep="\t")

tweets_df1 = tweets_df1[['text']]
tweets_df2 = tweets_df2[['text']]
tweets_df3 = tweets_df3[['text']]
tweets_df4 = tweets_df4[['text']]

tweets_df1.rename(columns={'text':'Tweets'},inplace=True)
tweets_df2.rename(columns={'text':'Tweets'},inplace=True)
tweets_df3.rename(columns={'text':'Tweets'},inplace=True)
tweets_df4.rename(columns={'text':'Tweets'},inplace=True)

# Give the topic
tweets_df1.loc[:, 'Topics'] = 'earthquakes'
tweets_df2.loc[:, 'Topics'] = 'floods'
tweets_df3.loc[:, 'Topics'] = 'forestfires'
tweets_df4.loc[:, 'Topics'] = 'hurricanes'

# Concatenate
tweets_df = tweets_df1
tweets_df = tweets_df.append(tweets_df2, ignore_index = True)
tweets_df = tweets_df.append(tweets_df3, ignore_index = True)
tweets_df = tweets_df.append(tweets_df4, ignore_index = True)

# Shuffle
tweets_df = tweets_df.sample(frac=1.0, random_state = 100).reset_index(drop=True)

'''

'\n# Former method\n\npath1 = \'/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/earthquakes_eyewitness_crowdflower_2000.tsv\'\npath2 = \'/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/floods_eyewitness_crowdflower_2000.tsv\'\npath3 = \'/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/forestfires_eyewitness_crowdflower_2000.tsv\'\npath4 = \'/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/hurricanes_eyewitness_crowdflower_2000.tsv\'\n\ntweets_df1=pd.read_csv(path1, sep="\t")\ntweets_df2=pd.read_csv(path2, sep="\t")\ntweets_df3=pd.read_csv(path3, sep="\t")\ntweets_df4=pd.read_csv(path4, sep="\t")\n\ntweets_df1 = tweets_df1[[\'text\']]\ntweets_df2 = tweets_df2[[\'text\']]\ntweets_df3 = tweets_df3[[\'text\']]\ntweets_df4 = tweets_df4[[\'text\']]\n\ntweets_df1.rename(columns={\'text\':\'Tweets\'},inplace=True)\ntweets_df2.rename(columns={\'text\':\'Tweets\'},inplace=True)\ntweets_df3.rename(columns={\'text\':\'Tweets

In [ ]:
# Shuffle again
tweets_df = tweets_df.sample(frac=1.0, random_state = 100).reset_index(drop=True)

In [ ]:
# See if the partition is correct
tweets_df.partition.value_counts()

train    8000
Name: partition, dtype: int64

In [ ]:
# Uncomment to see the merged (or unmerged) dataframes
tweets_df

,Tweets,partition,Topics
0,NASA Sees #Tropical #Cyclone #Debbie Approachi...,train,3
1,A small earthquake just happened,train,0
2,'Unprecedented' Wildfires Break Out in Norther...,train,2
3,earthquake in #Oakland just now,train,0
4,I don t know if my heart was beating too fast...,train,0
...,...,...,...
7995,Go Dubs #earthquake #bayarea,train,0
7996,Guess this explains why my last apartment cons...,train,1
7997,Climatologist explains why 'the conditions are...,train,2
7998,i could've had my braces off today but debbie ...,train,3


In [ ]:
# Make sure you run this one before other methods!

def to_lowercase(text):
    return text.lower()

#testing the function on a single sample for explaination
print(to_lowercase('IN CHINESE WE CALL CAPITALIZATION AS BIG WRITTING, IN GERMAN AS WELL.'))

#converting every row of the column into lower case 
tweets_df.Tweets=tweets_df.Tweets.apply(to_lowercase)

in chinese we call capitalization as big writting, in german as well.


In [ ]:
def standardize_accented_chars(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

#testing the function on a single sample for explaination
print(standardize_accented_chars('sómě words such as résumé, café, prótest, divorcé, coördinate, exposé, latté.'))

#standardizing accented characters for every row
tweets_df.Tweets=tweets_df.Tweets.apply(standardize_accented_chars)

some words such as resume, cafe, protest, divorce, coordinate, expose, latte.


In [ ]:
# Not a method, just to check how many tweets contain urls

def get_number_of_urls(documents):
    print("{:.2f}% of documents contain urls".format(sum
(documents.apply(lambda x:x.find('http'))>0)/len
(documents)*100))

# Passing the 'Tweets' column of the dataframe as the argument
print(get_number_of_urls(tweets_df.Tweets)) 

57.98% of documents contain urls
None


In [ ]:
def remove_url(text):
    return re.sub(r'https?:\S*', '', text)

#testing the function on a single sample for explaination
print(remove_url('using https://www.google.com/ as an example'))

#removing urls from every row
tweets_df.Tweets=tweets_df.Tweets.apply(remove_url)

using  as an example


In [ ]:
def expand_contractions(text):
    expanded_words = [] 
    for word in text.split():
       expanded_words.append(contractions.fix(word)) 
    return ' '.join(expanded_words)

#testing the function on a single sample for explaination
print(expand_contractions("Don't is the same as do not"))

#expanding contractions for every row
tweets_df.Tweets=tweets_df.Tweets.apply(expand_contractions)

Do not is the same as do not


In [ ]:
def remove_mentions_and_tags(text):
    text = re.sub(r'@\S*', '', text)
    return re.sub(r'#\S*', '', text)

#testing the function on a single sample for explaination
print(remove_mentions_and_tags('Some random @abc and #def'))

#removing mentions and tags from every row
tweets_df.Tweets=tweets_df.Tweets.apply(remove_mentions_and_tags)

Some random  and 


In [ ]:
def keep_only_alphabet(text):
    return re.sub(r'[^a-zA-Z]', ' ', text)

#testing the function on a single sample for explaination
print(keep_only_alphabet('Just a bit more $$processing required.Just a bit!!!'))

#for all the rows
tweets_df.Tweets=tweets_df.Tweets.apply(keep_only_alphabet)

Just a bit more   processing required Just a bit   


In [ ]:
def remove_stop_words(text):
  """
  Returns text without stop words
  """
  text = word_tokenize(text)
  word_list = []
  for word in text:
      if word not in stopwords.words('english') and word != 'rt':
          word_list.append(word)

  return ' '.join(word_list)

#testing the function on a single sample for explaination
print(remove_stop_words('rt Test this text to see which are stop words.'))

#removing stop-words and short words from every row
tweets_df.Tweets=tweets_df.Tweets.apply(remove_stop_words)

Test text see stop words .


In [ ]:
def lemmatize(text):
  lemmatizer = WordNetLemmatizer()
  text_str = word_tokenize(text)
  new_words = []

  for word in text_str:
    new_words.append(lemmatizer.lemmatize(word))
  return ' '.join(new_words)

#testing the function on a single sample for explaination
print(lemmatize('apples, bananas and pears are common fruits that are eaten by humans.'))

#Performing lemmatization on every row
tweets_df.Tweets=tweets_df.Tweets.apply(lemmatize)

apple , banana and pear are common fruit that are eaten by human .


In [ ]:
# Delete blank rows to fit OCTIS
tweets_df = tweets_df[tweets_df['Tweets'].str.len()>=1]
tweets_df.reset_index(inplace=True)

# Delete the index column caused by reset_index
del tweets_df['index']

# Delete tweets that only contain 1 or 2 words
for i in range(tweets_df.shape[0]):
    if len(tweets_df['Tweets'][i].split()) < 3:
        tweets_df.drop(index = i, inplace = True)

# (OPTIONAL) Delete tweets using the length of string
#tweets_df = tweets_df[tweets_df['Tweets'].str.len()>=20]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# Uncomment this to check how the df looks like after running
tweets_df

,Tweets,partition,Topics
0,nasa see approaching landfall nasa aqua,train,3
1,small earthquake happened,train,0
2,unprecedented wildfire break northern southern...,train,2
4,know heart beating fast felt earthquake,train,0
5,cyclone debbie coal mine hopeful reopening soo...,train,3
...,...,...,...
7983,turn completely flooded school closed car subm...,train,3
7985,guess explains last apartment consistently flo...,train,1
7986,climatologist explains condition primed fire c...,train,2
7987,could brace today debbie damage,train,3


In [ ]:
tweets_df.reset_index(inplace=True)
del tweets_df['index']

Make dataset and vocabulary for OCTIS

In [ ]:
# Create vocabulary.txt

def df_to_vocab(df):
    word_list = []
    for i in range(df.shape[0]):
        text = word_tokenize(df.Tweets[i])
        for word in text:
            if word not in word_list:
                word_list.append(word)
    return word_list

word_list = df_to_vocab(tweets_df)
len(word_list)

9266

In [ ]:
def list_to_vocab(word_list):
    txt = open(f"{save_dir}/vocabulary.txt", 'w')
    for i in range(len(word_list)):
        txt.write(word_list[i])
        txt.write('\r\n')
    txt.close()

list_to_vocab(word_list)

In [ ]:
tweets_df

,Tweets,partition,Topics
0,nasa see approaching landfall nasa aqua,train,3
1,small earthquake happened,train,0
2,unprecedented wildfire break northern southern...,train,2
3,know heart beating fast felt earthquake,train,0
4,cyclone debbie coal mine hopeful reopening soo...,train,3
...,...,...,...
7301,turn completely flooded school closed car subm...,train,3
7302,guess explains last apartment consistently flo...,train,1
7303,climatologist explains condition primed fire c...,train,2
7304,could brace today debbie damage,train,3


In [ ]:
#tweets_df.to_csv(f'{save_dir}/corpus.tsv',\
#                 sep = '\t', index=False, header = None)

In [ ]:
#tweets_df.to_csv(f'{save_dir}/corpus_with_header.tsv',\
#                 sep = '\t', index=False)

In [ ]:
# Use this as a check point
#assert False

# OCTIS Initiation

In [ ]:
!pip install octis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import octis
from octis.models.LDA import LDA
from octis.models.NMF import NMF
from octis.models.CTM import CTM
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
import numpy as np

In [ ]:
!pip install top2vec
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Save TC and TD
TCTD_line = []
TCTD_all = []

In [ ]:
%cd /content/drive/MyDrive/SS_2022_Praktikum

/content/drive/MyDrive/SS_2022_Praktikum


From here on, we output two matrixes and visualization for all the algorithms and datasets

# OCTIS -- LDA with crisis dataset 12

With our own data preprocessing method

In [ ]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder(save_dir)

In [ ]:
import time
begin_time = time.time()

model = LDA(num_topics = 4, random_state = 98)  # Create model
output_LDA_crisis = model.train_model(dataset) # Train the model

end_time = time.time()
run_time_LDA_crisis = round(end_time-begin_time, 2)

In [ ]:
# See what are in the model output

#print(*list(output_LDA_crisis.keys()), sep="\n")

In [ ]:
# We can delete it later
for t in output_LDA_crisis['topics'][:]:
  print(" ".join(t))

debbie cyclone flood wildfire ex queensland school fort australia rain
wildfire california flood fire people northern home right via debbie
flood debbie cyclone wildfire earthquake like smoke flash really phone
earthquake flood felt feel flash flooding cyclone debbie warning like


In [ ]:
# Initialize metric
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_v')

# Initialize metric
topic_diversity = TopicDiversity(topk=10)

In [ ]:
# Retrieve metrics score
topic_diversity_score = topic_diversity.score(output_LDA_crisis)
print("Topic diversity: " + str(topic_diversity_score))

npmi_score = npmi.score(output_LDA_crisis)
print("Coherence: " + str(npmi_score))

Topic diversity: 0.675
Coherence: 0.3164769956783974


In [ ]:
TCTD_line.append('Crisis_12')
TCTD_line.append('LDA')
TCTD_line.append(topic_diversity_score)
TCTD_line.append(npmi_score)
TCTD_all.append(TCTD_line)
TCTD_line = []

In [ ]:
# Match the number of labels to topic num id

# Dataset_12
in_labels_crisis_12 = {0:'earthquake',
                       1:'floods',
                       2:'forestfires',
                       3:'hurricanes'}

In [ ]:
# Doc-Topic Output

col_names_DT = ('run_id', 'Document ID', 'Document', 'Real Label', 'Assigned Topic Num', 'Assignment Score')
result_DT = []

# Dataset 12
df = pd.read_csv('/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/corpus_with_header.tsv', delimiter='\t')

# Crisis dataset
df = df.loc[df['partition'] == 'train']
output_max = np.max(output_LDA_crisis['topic-document-matrix'], axis = 0)
output_index = np.argmax(output_LDA_crisis['topic-document-matrix'], axis = 0)
length = len(output_max)
#print(df)

for i in range(length):
    result_DT_line = []

    result_DT_line.append(begin_time)
    result_DT_line.append(i)
    result_DT_line.append(df[i:i+1].Tweets.values[0])
    result_DT_line.append(in_labels_crisis_12[df[i:i+1].Topics.values[0]])

    result_DT_line.append(output_index[i])
    result_DT_line.append(output_max[i])

    result_DT.append(result_DT_line)

Doc_Topic_df = pd.DataFrame(result_DT, columns=col_names_DT)

Doc_Topic_df

,run_id,Document ID,Document,Real Label,Assigned Topic Num,Assignment Score
0,1.657398e+09,0,nasa see approaching landfall nasa aqua,hurricanes,0,0.888356
1,1.657398e+09,1,small earthquake happened,earthquake,3,0.811163
2,1.657398e+09,2,unprecedented wildfire break northern southern...,forestfires,1,0.891099
3,1.657398e+09,3,know heart beating fast felt earthquake,earthquake,1,0.545280
4,1.657398e+09,4,cyclone debbie coal mine hopeful reopening soo...,hurricanes,1,0.381486
...,...,...,...,...,...,...
7301,1.657398e+09,7301,turn completely flooded school closed car subm...,hurricanes,0,0.804138
7302,1.657398e+09,7302,guess explains last apartment consistently flo...,floods,1,0.888648
7303,1.657398e+09,7303,climatologist explains condition primed fire c...,forestfires,1,0.892635
7304,1.657398e+09,7304,could brace today debbie damage,hurricanes,0,0.624409


In [ ]:
# Topic-Word Output

col_names_TW = ("run_id", "method", "method_specific_params", "dataset", "num_given_topics", "reduced",
                "topic_num", "topic_size", "topic_words", "word_scores", "num_detected_topics",
                "num_final_topics", "duration_secs")
result_TW = []
size_count = Doc_Topic_df.loc[:, 'Assigned Topic Num'].value_counts()
topic_words = output_LDA_crisis['topics']

# Get word scores
scores = output_LDA_crisis['topic-word-matrix']

word_scores_all = []
for i in range(4):
    scores[i].sort()
    word_scores_line = scores[i][-10:][::-1]
    word_scores_all.append(word_scores_line)

for i in range(4):
    result_TW_line = []

    result_TW_line.append(begin_time)
    result_TW_line.append('LDA')
    result_TW_line.append('None')
    result_TW_line.append('crisis_12')
    result_TW_line.append(4)
    result_TW_line.append('None')
    result_TW_line.append(i)
    result_TW_line.append(size_count[i])
    result_TW_line.append(topic_words[i])
    result_TW_line.append(word_scores_all[i])
    result_TW_line.append('None')
    result_TW_line.append('None')
    result_TW_line.append(run_time_LDA_crisis)
    result_TW.append(result_TW_line)

Topic_word_df = pd.DataFrame(result_TW, columns=col_names_TW)
Topic_word_df

,run_id,method,method_specific_params,dataset,num_given_topics,reduced,topic_num,topic_size,topic_words,word_scores,num_detected_topics,num_final_topics,duration_secs
0,1.657398e+09,LDA,None,crisis_12,4,None,0,2093,"[debbie, cyclone, flood, wildfire, ex, queensl...","[0.04836821, 0.04828682, 0.027514843, 0.015727...",None,None,5.39
1,1.657398e+09,LDA,None,crisis_12,4,None,1,2015,"[wildfire, california, flood, fire, people, no...","[0.03956301, 0.03143632, 0.02617495, 0.0123464...",None,None,5.39
2,1.657398e+09,LDA,None,crisis_12,4,None,2,1158,"[flood, debbie, cyclone, wildfire, earthquake,...","[0.043155786, 0.014115971, 0.013581084, 0.0111...",None,None,5.39
3,1.657398e+09,LDA,None,crisis_12,4,None,3,2040,"[earthquake, flood, felt, feel, flash, floodin...","[0.06880951, 0.023676185, 0.013964915, 0.00984...",None,None,5.39


In [ ]:
from src import visualizer

visualizer.visualize_labels_per_topic(df_output_doc_topic=Doc_Topic_df,
                               df_output_topic_word=Topic_word_df,top_n_topics=None)

In [ ]:
visualizer.visualize_barchart(df_output_topic_word=Topic_word_df,n_words=5)

# OCTIS -- LDA with 20News dataset

In [ ]:
# Define dataset
dataset = Dataset()
dataset.fetch_dataset("20NewsGroup")

In [ ]:
# The length of testset
len(dataset.get_corpus()[13862:])

2447

In [ ]:
import time
begin_time = time.time()

model = LDA(num_topics=20, alpha=0.1, random_state = 100)   # Create Model
output_LDA_20news = model.train_model(dataset)    # Train the model

end_time = time.time()
run_time_LDA_20news = round(end_time-begin_time, 2)

In [ ]:
for t in output_LDA_20news['topics'][:5]:
  print(" ".join(t))

people time make child give religion man day thing word
people government gun state law weapon crime criminal kill make
people make time blue work ticket hand ground police thing
man homosexual make light people good turn leave time gay
key chip encryption make clipper post security people law public


In [ ]:
# Initialize metric
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_v')

# Initialize metric
topic_diversity = TopicDiversity(topk=10)

In [ ]:
# Retrieve metrics score
topic_diversity_score = topic_diversity.score(output_LDA_20news)
print("Topic diversity: " + str(topic_diversity_score))

npmi_score = npmi.score(output_LDA_20news)
print("Coherence: " + str(npmi_score))

Topic diversity: 0.695
Coherence: 0.5073820330016738


In [ ]:
TCTD_line.append('20_news')
TCTD_line.append('LDA')
TCTD_line.append(topic_diversity_score)
TCTD_line.append(npmi_score)
TCTD_all.append(TCTD_line)
TCTD_line = []

In [ ]:
# Use dataset.get_corpus()[13862:] if you would like to use the testset
# Here, we use training set only
orig_20news = dataset.get_corpus()[:11415]
all_20news = []
for i in range(len(orig_20news)):
    full = ' '.join(s for s in orig_20news[i])
    all_20news.append(full)
all_20news_labels = dataset.get_labels()

In [ ]:
# Doc-Topic Output

col_names_DT = ('Document ID', 'Document', 'Real Label', 'Assigned Topic Num', 'Assignment Score')
result_DT = []

# 20 news dataset
length = len(all_20news)
output_max = np.max(output_LDA_20news['topic-document-matrix'], axis = 0)
output_index = np.argmax(output_LDA_20news['topic-document-matrix'], axis = 0)

for i in range(length):
    result_DT_line = []

    result_DT_line.append(i)
    result_DT_line.append(all_20news[i])
    result_DT_line.append(all_20news_labels[i])
    result_DT_line.append(output_index[i])
    result_DT_line.append(output_max[i])

    result_DT.append(result_DT_line)

Doc_Topic_df = pd.DataFrame(result_DT, columns=col_names_DT)

Doc_Topic_df

,Document ID,Document,Real Label,Assigned Topic Num,Assignment Score
0,0,fax modem card sell mail,misc.forsale,10,0.728531
1,1,run server server install run add,comp.windows.x,12,0.668365
2,2,live part lead wait important remember judge j...,soc.religion.christian,0,0.499256
3,3,doesn pain deserve die lie rape,talk.religion.misc,0,0.538701
4,4,sale mile good condition good condition player...,rec.autos,13,0.676404
...,...,...,...,...,...
11410,11410,week half ago post people feeling family print...,comp.os.ms-windows.misc,6,0.417234
11411,11411,false claim wrong detail paragraph put quote w...,alt.atheism,17,0.551511
11412,11412,buy run afford put guy clue respond,comp.sys.mac.hardware,2,0.788835
11413,11413,match amazing red smoke screen point man argum...,talk.religion.misc,17,0.696580


In [ ]:
# Topic-Word Output

col_names_TW = ("run_id", "method", "method_specific_params", "dataset", "num_given_topics", "reduced",
                "topic_num", "topic_size", "topic_words", "word_scores", "num_detected_topics",
                "num_final_topics", "duration_secs")
result_TW = []
size_count = Doc_Topic_df.loc[:, 'Assigned Topic Num'].value_counts()
topic_words = output_LDA_20news['topics']

# Get word scores
scores = output_LDA_20news['topic-word-matrix']

word_scores_all = []
for i in range(20):
    scores[i].sort()
    word_scores_line = scores[i][-10:][::-1]
    word_scores_all.append(word_scores_line)

for i in range(20):
    result_TW_line = []

    result_TW_line.append(begin_time)
    result_TW_line.append('LDA')
    result_TW_line.append('None')
    result_TW_line.append('20news')
    result_TW_line.append(20)
    result_TW_line.append('None')
    result_TW_line.append(i)
    result_TW_line.append(size_count[i])
    result_TW_line.append(topic_words[i])
    result_TW_line.append(word_scores_all[i])
    result_TW_line.append('None')
    result_TW_line.append('None')
    result_TW_line.append(run_time_LDA_20news)

    result_TW.append(result_TW_line)

Topic_word_df = pd.DataFrame(result_TW, columns=col_names_TW)
Topic_word_df

,run_id,method,method_specific_params,dataset,num_given_topics,reduced,topic_num,topic_size,topic_words,word_scores,num_detected_topics,num_final_topics,duration_secs
0,1.657398e+09,LDA,None,20news,20,None,0,836,"[people, time, make, child, give, religion, ma...","[0.021425478, 0.012586859, 0.0110227605, 0.010...",None,None,19.23
1,1.657398e+09,LDA,None,20news,20,None,1,747,"[people, government, gun, state, law, weapon, ...","[0.023183126, 0.021261772, 0.011655791, 0.0111...",None,None,19.23
2,1.657398e+09,LDA,None,20news,20,None,2,314,"[people, make, time, blue, work, ticket, hand,...","[0.01685436, 0.013362161, 0.013286625, 0.01181...",None,None,19.23
3,1.657398e+09,LDA,None,20news,20,None,3,489,"[man, homosexual, make, light, people, good, t...","[0.017308336, 0.012387878, 0.010722764, 0.0090...",None,None,19.23
4,1.657398e+09,LDA,None,20news,20,None,4,533,"[key, chip, encryption, make, clipper, post, s...","[0.036257334, 0.016942162, 0.014701094, 0.0132...",None,None,19.23
5,1.657398e+09,LDA,None,20news,20,None,5,256,"[armenian, space, turkish, launch, year, russi...","[0.022938794, 0.01913609, 0.018803585, 0.01736...",None,None,19.23
6,1.657398e+09,LDA,None,20news,20,None,6,639,"[color, driver, bit, mode, card, run, work, di...","[0.018105589, 0.01807492, 0.016809972, 0.01520...",None,None,19.23
7,1.657398e+09,LDA,None,20news,20,None,7,181,"[page, title, text, include, information, dept...","[0.015317478, 0.014840858, 0.013197448, 0.0127...",None,None,19.23
8,1.657398e+09,LDA,None,20news,20,None,8,877,"[file, image, mail, program, information, incl...","[0.031733245, 0.01678779, 0.01574168, 0.014980...",None,None,19.23
9,1.657398e+09,LDA,None,20news,20,None,9,236,"[win, year, gun, make, team, rate, lose, back,...","[0.035007846, 0.021987418, 0.019811532, 0.0163...",None,None,19.23


In [ ]:
visualizer.visualize_labels_per_topic(df_output_doc_topic=Doc_Topic_df,
                               df_output_topic_word=Topic_word_df,top_n_topics=None)

In [ ]:
visualizer.visualize_barchart(df_output_topic_word=Topic_word_df,n_words=5)

# OCTIS -- NMF with crisis dataset 12

In [ ]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder(save_dir)

In [ ]:
import time
begin_time = time.time()

model = NMF(num_topics = 4, random_state = 96)  # Create model
output_NMF_crisis = model.train_model(dataset) # Train the model

end_time = time.time()
run_time_NMF_crisis = round(end_time-begin_time, 2)

In [ ]:
for t in output_NMF_crisis['topics'][:]:
  print(" ".join(t))

flood flash warning cyclone debbie rain watch area water going
earthquake flood felt like feel fire go twitter california first
wildfire california fort northern mcmurray canada smoke fire canadian people
cyclone debbie ex queensland school tropical australia rain south closed


In [ ]:
# Initialize metric
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_v')

# Initialize metric
topic_diversity = TopicDiversity(topk=10)

In [ ]:
# Retrieve metrics score
topic_diversity_score = topic_diversity.score(output_NMF_crisis)
print("Topic diversity: " + str(topic_diversity_score))

npmi_score = npmi.score(output_NMF_crisis)
print("Coherence: " + str(npmi_score))

Topic diversity: 0.85
Coherence: 0.49611643991760246


In [ ]:
TCTD_line.append('Crisis_12')
TCTD_line.append('NMF')
TCTD_line.append(topic_diversity_score)
TCTD_line.append(npmi_score)
TCTD_all.append(TCTD_line)
TCTD_line = []

In [ ]:
# Doc-Topic Output

col_names_DT = ('Document ID', 'Document', 'Real Label', 'Assigned Topic Num', 'Assignment Score')
result_DT = []

# Dataset 12
df = pd.read_csv('/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/corpus_with_header.tsv', delimiter='\t')

# Crisis dataset
df = df.loc[df['partition'] == 'train']
output_max = np.max(output_NMF_crisis['topic-document-matrix'], axis = 0)
output_index = np.argmax(output_NMF_crisis['topic-document-matrix'], axis = 0)
length = len(output_max)
#print(df)

for i in range(length):
    result_DT_line = []

    result_DT_line.append(i)
    result_DT_line.append(df[i:i+1].Tweets.values[0])
    result_DT_line.append(in_labels_crisis_12[df[i:i+1].Topics.values[0]])
    result_DT_line.append(output_index[i])
    result_DT_line.append(output_max[i])

    result_DT.append(result_DT_line)

Doc_Topic_df = pd.DataFrame(result_DT, columns=col_names_DT)

Doc_Topic_df

,Document ID,Document,Real Label,Assigned Topic Num,Assignment Score
0,0,nasa see approaching landfall nasa aqua,hurricanes,2,0.325869
1,1,small earthquake happened,earthquake,1,1.000000
2,2,unprecedented wildfire break northern southern...,forestfires,2,1.000000
3,3,know heart beating fast felt earthquake,earthquake,1,1.000000
4,4,cyclone debbie coal mine hopeful reopening soo...,hurricanes,3,0.955586
...,...,...,...,...,...
7301,7301,turn completely flooded school closed car subm...,hurricanes,3,0.992386
7302,7302,guess explains last apartment consistently flo...,floods,0,0.432808
7303,7303,climatologist explains condition primed fire c...,forestfires,2,0.814563
7304,7304,could brace today debbie damage,hurricanes,3,0.922867


In [ ]:
# Topic-Word Output

col_names_TW = ("run_id", "method", "method_specific_params", "dataset", "num_given_topics", "reduced",
                "topic_num", "topic_size", "topic_words", "word_scores", "num_detected_topics",
                "num_final_topics", "duration_secs")
result_TW = []
size_count = Doc_Topic_df.loc[:, 'Assigned Topic Num'].value_counts()
topic_words = output_NMF_crisis['topics']

# Get word scores
scores = output_NMF_crisis['topic-word-matrix']

word_scores_all = []
for i in range(4):
    scores[i].sort()
    word_scores_line = scores[i][-10:][::-1]
    word_scores_all.append(word_scores_line)

for i in range(4):
    result_TW_line = []

    result_TW_line.append(begin_time)
    result_TW_line.append('NMF')
    result_TW_line.append('None')
    result_TW_line.append('crisis_12')
    result_TW_line.append(4)
    result_TW_line.append('None')
    result_TW_line.append(i)
    result_TW_line.append(size_count[i])
    result_TW_line.append(topic_words[i])
    result_TW_line.append(word_scores_all[i])
    result_TW_line.append('None')
    result_TW_line.append('None')
    result_TW_line.append(run_time_NMF_crisis)
    result_TW.append(result_TW_line)

Topic_word_df = pd.DataFrame(result_TW, columns=col_names_TW)
Topic_word_df

,run_id,method,method_specific_params,dataset,num_given_topics,reduced,topic_num,topic_size,topic_words,word_scores,num_detected_topics,num_final_topics,duration_secs
0,1.657398e+09,NMF,None,crisis_12,4,None,0,2097,"[flood, flash, warning, cyclone, debbie, rain,...","[0.10286273294877718, 0.021460159384637828, 0....",None,None,6.01
1,1.657398e+09,NMF,None,crisis_12,4,None,1,1867,"[earthquake, flood, felt, like, feel, fire, go...","[0.09618331095879157, 0.025469058246500302, 0....",None,None,6.01
2,1.657398e+09,NMF,None,crisis_12,4,None,2,1836,"[wildfire, california, fort, northern, mcmurra...","[0.09458816807590659, 0.053048767061298796, 0....",None,None,6.01
3,1.657398e+09,NMF,None,crisis_12,4,None,3,1506,"[cyclone, debbie, ex, queensland, school, trop...","[0.10586460070591208, 0.10543682034857255, 0.0...",None,None,6.01


In [ ]:
visualizer.visualize_labels_per_topic(df_output_doc_topic=Doc_Topic_df,
                               df_output_topic_word=Topic_word_df,top_n_topics=None)

In [ ]:
visualizer.visualize_barchart(df_output_topic_word=Topic_word_df,n_words=5)

# OCTIS -- NMF with 20News dataset

In [ ]:
# Define dataset
dataset = Dataset()
dataset.fetch_dataset("20NewsGroup")

In [ ]:
import time
begin_time = time.time()

model = NMF(num_topics=20, random_state = 100)   # Create Model
output_NMF_20news = model.train_model(dataset)    # Train the model

end_time = time.time()
run_time_NMF_20news = round(end_time-begin_time, 2)

In [ ]:
for t in output_NMF_20news['topics'][:5]:
  print(" ".join(t))

server include base support send widget subject motif mail source
system user list information internet application post run window email
system key question atheist post argument exist belief true religion
graphic image mail send package format object datum model include
government turkish russian administration official support fund american war food


In [ ]:
# Initialize metric
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_v')

# Initialize metric
topic_diversity = TopicDiversity(topk=10)

In [ ]:
# Retrieve metrics score
topic_diversity_score = topic_diversity.score(output_NMF_20news)
print("Topic diversity: " + str(topic_diversity_score))

npmi_score = npmi.score(output_NMF_20news)
print("Coherence: " + str(npmi_score))

Topic diversity: 0.695
Coherence: 0.5978809563977465


In [ ]:
TCTD_line.append('20_news')
TCTD_line.append('NMF')
TCTD_line.append(topic_diversity_score)
TCTD_line.append(npmi_score)
TCTD_all.append(TCTD_line)
TCTD_line = []

In [ ]:
# Doc-Topic Output

col_names_DT = ('Document ID', 'Document', 'Real Label', 'Assigned Topic Num', 'Assignment Score')
result_DT = []

# 20 news dataset
length = len(all_20news)
output_max = np.max(output_NMF_20news['topic-document-matrix'], axis = 0)
output_index = np.argmax(output_NMF_20news['topic-document-matrix'], axis = 0)

for i in range(length):
    result_DT_line = []

    result_DT_line.append(i)
    result_DT_line.append(all_20news[i])
    result_DT_line.append(all_20news_labels[i])
    result_DT_line.append(output_index[i])
    result_DT_line.append(output_max[i])

    result_DT.append(result_DT_line)

Doc_Topic_df = pd.DataFrame(result_DT, columns=col_names_DT)

Doc_Topic_df

,Document ID,Document,Real Label,Assigned Topic Num,Assignment Score
0,0,fax modem card sell mail,misc.forsale,3,0.479223
1,1,run server server install run add,comp.windows.x,0,0.656878
2,2,live part lead wait important remember judge j...,soc.religion.christian,15,0.634671
3,3,doesn pain deserve die lie rape,talk.religion.misc,17,0.373880
4,4,sale mile good condition good condition player...,rec.autos,9,0.218945
...,...,...,...,...,...
11410,11410,week half ago post people feeling family print...,comp.os.ms-windows.misc,9,0.236355
11411,11411,false claim wrong detail paragraph put quote w...,alt.atheism,2,0.583568
11412,11412,buy run afford put guy clue respond,comp.sys.mac.hardware,17,0.422707
11413,11413,match amazing red smoke screen point man argum...,talk.religion.misc,2,0.488682


In [ ]:
# Topic-Word Output

col_names_TW = ("run_id", "method", "method_specific_params", "dataset", "num_given_topics", "reduced",
                "topic_num", "topic_size", "topic_words", "word_scores", "num_detected_topics",
                "num_final_topics", "duration_secs")
result_TW = []
size_count = Doc_Topic_df.loc[:, 'Assigned Topic Num'].value_counts()
topic_words = output_NMF_20news['topics']

# Get word scores
scores = output_NMF_20news['topic-word-matrix']

word_scores_all = []
for i in range(20):
    scores[i].sort()
    word_scores_line = scores[i][-10:][::-1]
    word_scores_all.append(word_scores_line)

for i in range(20):
    result_TW_line = []

    result_TW_line.append(begin_time)
    result_TW_line.append('NMF')
    result_TW_line.append('None')
    result_TW_line.append('20news')
    result_TW_line.append(20)
    result_TW_line.append('None')
    result_TW_line.append(i)
    result_TW_line.append(size_count[i])
    result_TW_line.append(topic_words[i])
    result_TW_line.append(word_scores_all[i])
    result_TW_line.append('None')
    result_TW_line.append('None')
    result_TW_line.append(run_time_NMF_20news)

    result_TW.append(result_TW_line)

Topic_word_df = pd.DataFrame(result_TW, columns=col_names_TW)
Topic_word_df

,run_id,method,method_specific_params,dataset,num_given_topics,reduced,topic_num,topic_size,topic_words,word_scores,num_detected_topics,num_final_topics,duration_secs
0,1.657398e+09,NMF,None,20news,20,None,0,304,"[server, include, base, support, send, widget,...","[0.047522853189604246, 0.027720347156302994, 0...",None,None,11.46
1,1.657398e+09,NMF,None,20news,20,None,1,581,"[system, user, list, information, internet, ap...","[0.03994182210814352, 0.023456913968699443, 0....",None,None,11.46
2,1.657398e+09,NMF,None,20news,20,None,2,949,"[system, key, question, atheist, post, argumen...","[0.021057689825443017, 0.01873972495004804, 0....",None,None,11.46
3,1.657398e+09,NMF,None,20news,20,None,3,315,"[graphic, image, mail, send, package, format, ...","[0.05038194293396061, 0.036369582402145655, 0....",None,None,11.46
4,1.657398e+09,NMF,None,20news,20,None,4,298,"[government, turkish, russian, administration,...","[0.052614212748131155, 0.0400045579440005, 0.0...",None,None,11.46
5,1.657398e+09,NMF,None,20news,20,None,5,419,"[window, widget, call, application, subject, s...","[0.05689365757699758, 0.029058052537795098, 0....",None,None,11.46
6,1.657398e+09,NMF,None,20news,20,None,6,172,"[privacy, key, internet, encryption, computer,...","[0.03020465162385621, 0.027352834902245046, 0....",None,None,11.46
7,1.657398e+09,NMF,None,20news,20,None,7,1286,"[game, team, season, play, player, year, win, ...","[0.0403570490585354, 0.0358469716154559, 0.026...",None,None,11.46
8,1.657398e+09,NMF,None,20news,20,None,8,341,"[launch, planet, year, space, earth, satellite...","[0.03076726575584797, 0.030159306503787522, 0....",None,None,11.46
9,1.657398e+09,NMF,None,20news,20,None,9,1394,"[program, question, make, work, president, pro...","[0.04392107761527465, 0.03099353406130468, 0.0...",None,None,11.46


In [ ]:
visualizer.visualize_labels_per_topic(df_output_doc_topic=Doc_Topic_df,
                               df_output_topic_word=Topic_word_df,top_n_topics=None)

In [ ]:
visualizer.visualize_barchart(df_output_topic_word=Topic_word_df,n_words=5)

# OCTIS -- CTM with crisis dataset 12

In [ ]:
assert False

AssertionError: ignored

In [ ]:
dataset = Dataset()
dataset.fetch_dataset("20NewsGroup")

In [ ]:
import time
begin_time = time.time()

model = CTM(num_topics=20)   # Create Model
output_CTM_20news = model.train_model(dataset)    # Train the model

end_time = time.time()
run_time_CTM_20news = round(end_time-begin_time, 2)

In [ ]:
# We can delete it later
for t in output_LDA_crisis['topics'][:]:
  print(" ".join(t))

In [ ]:
# Initialize metric
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_v')

# Initialize metric
topic_diversity = TopicDiversity(topk=10)

In [ ]:
# Retrieve metrics score
topic_diversity_score = topic_diversity.score(output_LDA_crisis)
print("Topic diversity: " + str(topic_diversity_score))

npmi_score = npmi.score(output_LDA_crisis)
print("Coherence: " + str(npmi_score))

In [ ]:
TCTD_line.append('Crisis_12')
TCTD_line.append('CTM')
TCTD_line.append(topic_diversity_score)
TCTD_line.append(npmi_score)
TCTD_all.append(TCTD_line)
TCTD_line = []

# Evaluation

In [ ]:
# TC and TD
col_names_TCTD = ("Dataset", "algorithm", "Topic Diversity", "Topic Coherence")
TCTD_df = pd.DataFrame(TCTD_all, columns=col_names_TCTD)

TCTD_df

In [ ]:
assert False

In [ ]:
a = dataset._Dataset__corpus[0]

In [ ]:
len(dataset.get_labels())

In [ ]:
# Match the number of labels to topics
# Adjust according to the output in the block below

# Dataset_12
in_labels = {0:'earthquake',
             1:'floods',
             2:'forestfires',
             3:'hurricanes'}

out_labels = {0:'hurricanes',
              1:'earthquake',
              2:'floods',
              3:'forestfires'}

# 20 news dataset (Not accurate, we don't use this for now)
#labels_20news = {}
#for i in range(20):
#    labels_20news[i] = dataset._Dataset__metadata['labels'][i]

In [ ]:
# Make 20 news test dataset
# testset_20news: The texts
# testset_20news_labels: The labels
dataset = Dataset()
dataset.fetch_dataset("20NewsGroup")

# Use dataset.get_corpus()[13862:] if you would like to use the testset
orig_20news = dataset.get_corpus()[:11415]
all_20news = []
for i in range(len(orig_20news)):
    full = ' '.join(s for s in orig_20news[i])
    all_20news.append(full)
all_20news_labels = dataset.get_labels()

In [ ]:
for t in output_NMF_crisis['topics'][:]:
  print(" ".join(t))

In [ ]:
np.shape(output_NMF_20news['topic-document-matrix'])

In [ ]:
# Doc-Topic Output (Only test set here)
# Why do we need "run_id"
# 4 changes to make in total when changing the algorithm (and the corresponding dataset)
# 1 change to make when changing the dataset (by commenting)

col_names_DT = ("Document", "Real Label", "Assigned Topic Num", "Assignment Score")
result_DT = []

# Dataset 1
#df = pd.read_csv('/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_1/corpus_with_header.tsv', delimiter='\t')

# Dataset 12
#df = pd.read_csv('/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/corpus_with_header.tsv', delimiter='\t')

# Crisis dataset
#df = df.loc[df['partition'] == 'test']
#output_max = np.max(output_NMF_crisis['topic-document-matrix'], axis = 0)
#output_index = np.argmax(output_NMF_crisis['topic-document-matrix'], axis = 0)
#length = len(output_max)

# 20 news dataset
length = len(all_20news)
output_max = np.max(output_LDA_20news['topic-document-matrix'], axis = 0)
output_index = np.argmax(output_LDA_20news['topic-document-matrix'], axis = 0)

for i in range(length):
    result_DT_line = []

    # Crisis dataset
    #result_DT_line.append(df[i:i+1].Tweets.values[0])
    #result_DT_line.append(in_labels[df[i:i+1].Topics.values[0]])

    # 20 news dataset
    result_DT_line.append(all_20news[i])
    result_DT_line.append(all_20news_labels[i])

    result_DT_line.append(output_index[i])
    result_DT_line.append(output_max[i])

    result_DT.append(result_DT_line)

Doc_Topic_df = pd.DataFrame(result_DT, columns=col_names_DT)

Doc_Topic_df

In [ ]:
# Method from Berk's work, 3q!
# Define length according to the dataset
length = 20

for i in range(length):
    print(f'Topic {i}:')
    print(Doc_Topic_df.query(f'`Assigned Topic Num` == {i}') ['Real Label'].value_counts())
    print('-'*32)

In [ ]:
# Topic-Word Output
# 1 change to make in total when changing the algorithm (and the corresponding dataset)
# 1 change to make when changing the dataset (by commenting)
# Remember to change NUM_TOPICS, it's also possible to change it here
NUM_TOPICS = 20

col_names_TW = ("num_given_topics", "topic_num", "topic_words")
result_TW = []

for i in range(NUM_TOPICS):
    result_TW_line = []

    result_TW_line.append(NUM_TOPICS)
    result_TW_line.append(i)
    result_TW_line.append(output_NMF_20news['topics'][i])
    result_TW.append(result_TW_line)

Topic_word_df = pd.DataFrame(result_TW, columns=col_names_TW)
Topic_word_df

In [ ]:
word_list.index('window')

In [ ]:
output_LDA_20news['topics']

In [ ]:
len(word_list)

Test (Delete when finished)

In [ ]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder(save_dir)
# Create Model
model = NMF(num_topics=4)

# Train the model
output = model.train_model(dataset) 

In [ ]:
#np.shape(output['topic-word-matrix'])
#output['topic-word-matrix']
print(np.max(output['topic-word-matrix'], axis = 0))

Note the 4 outputs:


*   *topics*: the list of word topics
*   *topic-word-matrix*: the distribution of the words of the vocabulary for each topic (dimensions: |num topics| x |vocabulary|)
*   *topic-document-matrix*: the distribution of the topics for each document of the training set (dimensions: |num topics| x |training documents|)
*   *test-topic-document-matrix*: the distribution of the topics for each document of the testing set (dimensions: |num topics| x |test documents|)


In [ ]:
print(*list(output.keys()), sep="\n")

In [ ]:
import numpy as np
#output_LDA_20news
output_LDA_20news['test-topic-document-matrix']

In [ ]:
output['test-topic-document-matrix']
output_max = np.max(output['test-topic-document-matrix'], axis = 0)
output_index = np.argmax(output['test-topic-document-matrix'], axis = 0)

output_index

In [ ]:
output

In [ ]:
# Archive
# Topic-Word Output

col_names_TW = ("run_id", "method", "method_specific_params", "dataset", "num_given_topics", "reduced",
                "topic_num", "topic_size", "topic_words", "word_scores", "num_detected_topics",
                "num_final_topics", "duration_secs")
result_TW = []
size_count = Doc_Topic_df.loc[:, 'Assigned Topic Num'].value_counts()
topic_words = output_LDA_crisis['topics']

# Get word scores
scores = output_LDA_crisis['topic-word-matrix']
word_list = []
with open('/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/vocabulary.txt') as f:
    for line in f:
        word_list.append(line[:-1])

word_scores_all = []
for i in range(4):
    word_scores_line = []
    for j in range(10):     # 10 is the topk, we always use 10 in this task
        word = topic_words[i][j]
        word_scores = scores[i][word_list.index(word)]
        word_scores_line.append(word_scores)
    word_scores_all.append(word_scores_line)

for i in range(4):
    result_TW_line = []

    result_TW_line.append(begin_time)
    result_TW_line.append('LDA')
    result_TW_line.append('None')
    result_TW_line.append('crisis_12')
    result_TW_line.append(4)
    result_TW_line.append('None')
    result_TW_line.append(i)
    result_TW_line.append(size_count[i])
    result_TW_line.append(topic_words[i])
    result_TW_line.append(word_scores_all[i])
    result_TW_line.append('None')
    result_TW_line.append('None')
    result_TW_line.append(run_time_LDA_crisis)
    result_TW.append(result_TW_line)

Topic_word_df = pd.DataFrame(result_TW, columns=col_names_TW)
Topic_word_df